# Práctica 5: Análisis de Imagen

Pipeline: carga, preprocesado (iluminación, filtros), segmentación, medidas (bordes/áreas), validación.

In [ ]:
# Imports básicos
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, filters, exposure, feature, morphology, measure
import cv2

In [ ]:
# Configuración
from pathlib import Path
DATA_DIR = Path('..') / 'data'
# Cambia el nombre de la imagen si es necesario
image_path = DATA_DIR / 'example.png'
print('Usando imagen:', image_path)

In [ ]:
# Carga y conversión a escala de grises
img = io.imread(image_path)
if img.ndim == 3:
    gray = color.rgb2gray(img)
else:
    gray = img.astype(float)/255.0
plt.imshow(gray, cmap='gray'); plt.title('Gray'); plt.axis('off')

In [ ]:
# Corrección de iluminación + suavizado
gray_eq = exposure.equalize_adapthist(gray, clip_limit=0.03)
gray_blur = cv2.GaussianBlur((gray_eq*255).astype('uint8'), (5,5), 0)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.imshow(gray_eq, cmap='gray'); plt.title('CLAHE'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(gray_blur, cmap='gray'); plt.title('Gaussian'); plt.axis('off')

In [ ]:
# Segmentación: Otsu y morfología
th = filters.threshold_otsu(gray_blur)
binary = gray_blur > th
binary = morphology.remove_small_holes(binary, area_threshold=64)
binary = morphology.remove_small_objects(binary, min_size=64)
plt.imshow(binary, cmap='gray'); plt.title('Binary'); plt.axis('off')

In [ ]:
# Medidas: contornos, áreas, centroides
labels = measure.label(binary)
props = measure.regionprops(labels)
print('Objetos:', len(props))
for i,p in enumerate(props[:5]):
    print(i, 'area', p.area, 'centroid', p.centroid)